<a href="https://colab.research.google.com/github/gianmarcomejia96/data-analytics-portfolio-gianmarcomejia/blob/main/perdidacrediticiaesperada_demo/p_01_perdidacrediticiaesperada_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import pandas as pd

In [85]:
#Leer dataset del proyecto
url = "https://raw.githubusercontent.com/gianmarcomejia96/data-analytics-portfolio-gianmarcomejia/main/perdidacrediticiaesperada_demo/cxc_auditoria_historico_3y.csv" #data sintética histórica de facturas de CxC comerciales"
df = pd.read_csv(url, encoding="latin1", sep=",", low_memory=False)

In [91]:
df = df.iloc[:, :13]
df.head(5)

,Periodo,Factura,Cliente,Fecha_Emision,Plazo_Pago_Dias,Fecha_Vencimiento,Historial_Morosidad,Antiguedad_Cliente_Anios,Saldo_Pendiente,Linea_Credito,Garantia,Incumplio,perdida_incumplimiento
0,2022-09,F00000001,C00179,16/09/2022,30,16/10/2022,NO,4.0,12251.38,17556.41,SI,SI,0.4
1,2022-09,F00000002,C01548,22/09/2022,45,6/11/2022,NO,6.5,40580.18,74804.84,NO,SI,0.6
2,2022-09,F00000003,C01309,27/09/2022,45,11/11/2022,NO,6.1,6440.01,11375.65,NO,SI,0.6
3,2022-09,F00000004,C00878,4/09/2022,60,3/11/2022,NO,6.9,20722.41,32190.75,NO,SI,0.6
4,2022-09,F00000005,C00866,5/09/2022,90,4/12/2022,SI,1.9,9467.73,12403.09,SI,SI,0.4


In [92]:
df.info() # información para ver todas las cabeceras, tipo de dato  si tiene algun nulo.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54000 entries, 0 to 53999
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Periodo                   54000 non-null  object 
 1   Factura                   54000 non-null  object 
 2   Cliente                   54000 non-null  object 
 3   Fecha_Emision             54000 non-null  object 
 4   Plazo_Pago_Dias           54000 non-null  int64  
 5   Fecha_Vencimiento         54000 non-null  object 
 6   Historial_Morosidad       54000 non-null  object 
 7   Antiguedad_Cliente_Anios  54000 non-null  float64
 8   Saldo_Pendiente           54000 non-null  float64
 9   Linea_Credito             54000 non-null  float64
 10  Garantia                  54000 non-null  object 
 11  Incumplio                 54000 non-null  object 
 12  perdida_incumplimiento    54000 non-null  float64
dtypes: float64(4), int64(1), object(8)
memory usage: 5.4+ MB


In [93]:
#Pérdida Crediticia Esperada (PCE)
# Qué es: un cálculo que estima cuánto dinero probablemente no se cobrará de clientes o de otros instrumentos financieros.
#Para qué sirve: registrar de forma anticipada una provisión que refleje el riesgo real de incobrabilidad.
#Fórmula: PCE = Saldo x probabilidad de incumplimiento (PI) x pérdida en caso de incumplimiento (PCI)

#Tenemos Saldo(Saldo_Pendiente) y PCI(50% flat en la mayoría de casos), entonces PI es un cálculo que se hace bajo NIIF 9(método de matriz de provisión.

print('La propuesta es hacer el cálculo clásico, luego proponer un modelo de regresión logística para estimar el PI y comparar resultados')


La propuesta es hacer el cálculo clásico, luego proponer un modelo de regresión logística para estimar el PI y comparar resultados


In [94]:
# establecer fechas de trabajo y todas los fields en fecha
fecha_corte = pd.Timestamp.today().normalize() # Hoy()
for col in ["Fecha_Emision", "Fecha_Vencimiento"]:
    df[col] = pd.to_datetime(df[col], dayfirst=True, errors="coerce")  # dayfirst=True por formato DD/MM/AAAA

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54000 entries, 0 to 53999
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Periodo                   54000 non-null  object        
 1   Factura                   54000 non-null  object        
 2   Cliente                   54000 non-null  object        
 3   Fecha_Emision             54000 non-null  datetime64[ns]
 4   Plazo_Pago_Dias           54000 non-null  int64         
 5   Fecha_Vencimiento         54000 non-null  datetime64[ns]
 6   Historial_Morosidad       54000 non-null  object        
 7   Antiguedad_Cliente_Anios  54000 non-null  float64       
 8   Saldo_Pendiente           54000 non-null  float64       
 9   Linea_Credito             54000 non-null  float64       
 10  Garantia                  54000 non-null  object        
 11  Incumplio                 54000 non-null  object        
 12  perdida_incumplimi

In [96]:
#Cálculo PCE clásico
# 1. Agrupar saldos por rango de días de vencimiento (aging buckets):

# 1.1 usamos el número de días desde el vencimiento
df["dias_vencidos"] = (
    (fecha_corte - df["Fecha_Vencimiento"]).dt.days
    .clip(lower=0)
    .fillna(0)
    .astype(int)
)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54000 entries, 0 to 53999
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Periodo                   54000 non-null  object        
 1   Factura                   54000 non-null  object        
 2   Cliente                   54000 non-null  object        
 3   Fecha_Emision             54000 non-null  datetime64[ns]
 4   Plazo_Pago_Dias           54000 non-null  int64         
 5   Fecha_Vencimiento         54000 non-null  datetime64[ns]
 6   Historial_Morosidad       54000 non-null  object        
 7   Antiguedad_Cliente_Anios  54000 non-null  float64       
 8   Saldo_Pendiente           54000 non-null  float64       
 9   Linea_Credito             54000 non-null  float64       
 10  Garantia                  54000 non-null  object        
 11  Incumplio                 54000 non-null  object        
 12  perdida_incumplimi

In [101]:
df.isnull().sum() #validador que no existen nulos.

,0
Periodo,0
Factura,0
Cliente,0
Fecha_Emision,0
Plazo_Pago_Dias,0
Fecha_Vencimiento,0
Historial_Morosidad,0
Antiguedad_Cliente_Anios,0
Saldo_Pendiente,0
Linea_Credito,0


Cálculo clásico bajo NIIF 9 dePérdida Crediticia Esperada (PCE)

In [125]:
# 1.2 Creamos los rangos o aging buckets:

def aging_bucket(dias_vencidos: int) -> str:
    if pd.isna(dias_vencidos): return "sin_fecha"
    if dias_vencidos <= 30:   return "0-30 días"
    if dias_vencidos <= 60:   return "31-60 días"
    if dias_vencidos <= 90:   return "61-90 días"
    if dias_vencidos <= 120:  return "91-120 días"
    if dias_vencidos <= 180:  return "121-180 días"
    if dias_vencidos <= 270:  return "181-270 días"
    if dias_vencidos <= 360:  return "271-360 días"
    return ">360 días"

df["rango"] = df["dias_vencidos"].apply(aging_bucket)

orden_rangos = [
    "0-30 días",
    "31-60 días",
    "61-90 días",
    "91-120 días",
    "121-180 días",
    "181-270 días",
    "271-360 días",
    ">360 días"
]

df["rango"] = pd.Categorical(df["rango"], categories=orden_rangos, ordered=True)

,rango
0,>360 días
1,>360 días
2,>360 días
3,>360 días
4,>360 días
...,...
53995,0-30 días
53996,0-30 días
53997,0-30 días
53998,0-30 días


In [176]:
# 2. Calcular la proporción histórica de incumplimientos por bucket

# Generamos una especie de pivot table para calcular % de saldo que no cumplió sobre el total del saldo por rango.
df_tabla = (
    df.groupby("rango", as_index=False)
      .agg(
          saldo_total_rango=("Saldo_Pendiente", "sum"),
          saldo_si_rango=("Saldo_Pendiente",
                          lambda x: x[df.loc[x.index, "Incumplio"].str.upper() == "SI"].sum())
      )
)

df_tabla["PD_rango"] = df_tabla["saldo_si_rango"] / df_tabla["saldo_total_rango"]

df_tabla["saldo_total_rango"] = df_tabla["saldo_total_rango"].apply(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
df_tabla["saldo_si_rango"] = df_tabla["saldo_si_rango"].apply(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

df_tabla


/tmp/ipython-input-1831052075.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rango", as_index=False)


,rango,saldo_total_rango,saldo_si_rango,PD_rango
0,0-30 días,"40.096.086,24","3.886.483,97",0.096929
1,31-60 días,"18.698.267,00","2.758.317,56",0.147517
2,61-90 días,"17.391.438,87","2.834.048,81",0.162957
3,91-120 días,"18.316.771,53","3.137.290,39",0.171280
4,121-180 días,"36.995.148,55","9.107.343,86",0.246177
5,181-270 días,"53.577.552,32","16.021.162,05",0.299028
6,271-360 días,"54.140.669,58","21.958.345,52",0.405579
7,>360 días,"422.263.315,53","278.390.080,21",0.659281


In [172]:
#3. Asignar esa PD a los saldos actuales en cada bucket
df = df.merge(df_tabla[["rango", "PD_rango"]], on="rango", how="left")


In [173]:
df.head(5)

,Periodo,Factura,Cliente,Fecha_Emision,Plazo_Pago_Dias,Fecha_Vencimiento,Historial_Morosidad,Antiguedad_Cliente_Anios,Saldo_Pendiente,Linea_Credito,Garantia,Incumplio,perdida_incumplimiento,dias_vencidos,rango,PD_rango_x,calculo_clasico,PD_rango_y
0,2022-09,F00000001,C00179,2022-09-16,30,2022-10-16,NO,4.0,12251.38,17556.41,SI,SI,0.4,1051,>360 días,0.659281,3230.839654,0.659281
1,2022-09,F00000002,C01548,2022-09-22,45,2022-11-06,NO,6.5,40580.18,74804.84,NO,SI,0.6,1030,>360 días,0.659281,16052.239183,0.659281
2,2022-09,F00000003,C01309,2022-09-27,45,2022-11-11,NO,6.1,6440.01,11375.65,NO,SI,0.6,1025,>360 días,0.659281,2547.464818,0.659281
3,2022-09,F00000004,C00878,2022-09-04,60,2022-11-03,NO,6.9,20722.41,32190.75,NO,SI,0.6,1033,>360 días,0.659281,8197.131747,0.659281
4,2022-09,F00000005,C00866,2022-09-05,90,2022-12-04,SI,1.9,9467.73,12403.09,SI,SI,0.4,1002,>360 días,0.659281,2496.756898,0.659281


In [151]:
#4. Calculo por fórmula por línea

df['calculo_clasico'] = df.Saldo_Pendiente*df.PD_rango*df.perdida_incumplimiento
df['calculo_clasico']
df.head(5)


,Periodo,Factura,Cliente,Fecha_Emision,Plazo_Pago_Dias,Fecha_Vencimiento,Historial_Morosidad,Antiguedad_Cliente_Anios,Saldo_Pendiente,Linea_Credito,Garantia,Incumplio,perdida_incumplimiento,dias_vencidos,rango,PD_rango,calculo_clasico
0,2022-09,F00000001,C00179,2022-09-16,30,2022-10-16,NO,4.0,12251.38,17556.41,SI,SI,0.4,1051,>360 días,0.659281,3230.839654
1,2022-09,F00000002,C01548,2022-09-22,45,2022-11-06,NO,6.5,40580.18,74804.84,NO,SI,0.6,1030,>360 días,0.659281,16052.239183
2,2022-09,F00000003,C01309,2022-09-27,45,2022-11-11,NO,6.1,6440.01,11375.65,NO,SI,0.6,1025,>360 días,0.659281,2547.464818
3,2022-09,F00000004,C00878,2022-09-04,60,2022-11-03,NO,6.9,20722.41,32190.75,NO,SI,0.6,1033,>360 días,0.659281,8197.131747
4,2022-09,F00000005,C00866,2022-09-05,90,2022-12-04,SI,1.9,9467.73,12403.09,SI,SI,0.4,1002,>360 días,0.659281,2496.756898


In [177]:
total_general = df["calculo_clasico"].sum()
totales_por_rango = df.groupby("rango", as_index = False,).agg(saldo_total=("Saldo_Pendiente", "sum"),pce_total=("calculo_clasico", "sum"))

totales_por_rango["saldo_total"] = totales_por_rango["saldo_total"].apply(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))
totales_por_rango["pce_total"] = totales_por_rango["pce_total"].apply(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

print(totales_por_rango)


          rango     saldo_total       pce_total
0     0-30 días   40.096.086,24    1.975.458,66
1    31-60 días   18.698.267,00    1.397.391,50
2    61-90 días   17.391.438,87    1.431.183,71
3   91-120 días   18.316.771,53    1.606.400,68
4  121-180 días   36.995.148,55    4.649.973,96
5  181-270 días   53.577.552,32    8.185.340,26
6  271-360 días   54.140.669,58   11.184.662,66
7     >360 días  422.263.315,53  141.788.683,80


/tmp/ipython-input-1739066894.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  totales_por_rango = df.groupby("rango", as_index = False,).agg(saldo_total=("Saldo_Pendiente", "sum"),pce_total=("calculo_clasico", "sum"))


In [178]:
totales_por_rango = pd.pivot_table(
    df,
    index="rango",
    values=["Saldo_Pendiente", "calculo_clasico"],
    aggfunc="sum",
    margins=True,           # agrega total general
    margins_name="TOTAL"    # nombre de la fila total
).reset_index()

# Función para formato español (punto miles, coma decimal)
def formato_espanol(x):
    return f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

# Aplicar formato a las columnas numéricas
for col in ["Saldo_Pendiente", "calculo_clasico"]:
    totales_por_rango[col] = totales_por_rango[col].apply(formato_espanol)

print(totales_por_rango)


          rango Saldo_Pendiente calculo_clasico
0     0-30 días   40.096.086,24    1.975.458,66
1    31-60 días   18.698.267,00    1.397.391,50
2    61-90 días   17.391.438,87    1.431.183,71
3   91-120 días   18.316.771,53    1.606.400,68
4  121-180 días   36.995.148,55    4.649.973,96
5  181-270 días   53.577.552,32    8.185.340,26
6  271-360 días   54.140.669,58   11.184.662,66
7     >360 días  422.263.315,53  141.788.683,80
8         TOTAL  661.479.249,62  172.219.095,23


/tmp/ipython-input-3433218157.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  totales_por_rango = pd.pivot_table(


Cálculo propuesta con modelo de regresión logística de Pérdida Crediticia Esperada (PCE) alineado con NIIF 9